## 데이터 수집

- https://kadx.co.kr/opmk/frn/pmumkproductDetail/PMU_79c6f1a4-56dd-492e-ad67-c5acba0304d2/5# 에서 무료 제공
- csv파일 3개 다운로드

## 데이터 전처리

### json file 관련 함수 

- save_json : 데이터를 json으로 저장
- load_json : json 파일 불러오기
- 기본 경로 현재 폴더인 'data'로 지정 해둠.

In [4]:
import os
import json

def save_json(data, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=4, ensure_ascii=False)

def load_json(filename):
    if not os.path.exists(filename):
        return None
    with open(filename, 'r', encoding='utf-8') as f:
        return json.load(f)

### 인코딩 
- 인코딩 utf-8

In [ ]:
import pandas as pd
import chardet

# 기존 파일 리스트
original_paths = [
    "TB_RECIPE_SEARCH-220701.csv",
    "TB_RECIPE_SEARCH-20231130.csv",
    "TB_RECIPE_SEARCH_241226.csv"
]

def encoding(filepath, num_bytes=10000):
    # 인코딩 감지
    with open(filepath, 'rb') as f:
        raw = f.read(num_bytes)
    result = chardet.detect(raw)
    return result['encoding']

# 인코딩 변환 및 덮어쓰기 저장
for path in original_paths:
    try:
        detected_encoding = encoding(path)
        print(f"[감지됨] {path} → 인코딩: {detected_encoding}")

        # 감지된 인코딩으로 열기
        with open(path, 'r', encoding=detected_encoding, errors='replace') as f:
            df = pd.read_csv(f)

        # UTF-8 인코딩 덮어쓰기 저장
        df.to_csv(path, index=False, encoding='utf-8-sig')

        print(f"[저장 완료] {path} → UTF-8 인코딩 완료")

    except Exception as e:
        print(f"[실패] {path}: {e}")

[감지됨] TB_RECIPE_SEARCH_241226.csv → 인코딩: utf-8
[저장 완료] TB_RECIPE_SEARCH_241226.csv → UTF-8 인코딩 완료


### 파일 통합
- 3개 파일 통합
- .json로 변환
- github push 용량 제한으로 다시 4분할

In [1]:
import pandas as pd
import numpy as np

# 파일 리스트
csv_files = [
    "TB_RECIPE_SEARCH-220701.csv",
    "TB_RECIPE_SEARCH-20231130.csv",
    "TB_RECIPE_SEARCH_241226.csv"
]

# 하나의 DataFrame으로 결합
df_list = []
for file in csv_files:
    try:
        df = pd.read_csv(file, encoding='utf-8-sig')
        df_list.append(df)
        print(f"[읽기 완료] {file}")
    except Exception as e:
        print(f"[오류] {file}: {e}")

# DataFrame 병합
merged_df = pd.concat(df_list, ignore_index=True)

# 4분할 저장
chunks = np.array_split(merged_df, 4) 
for i, chunk in enumerate(chunks, 1):
    output_file = f"recipe_mangae_{i}.json"
    chunk.to_json(output_file, orient='records', force_ascii=False, indent=2)
    print(f"[저장 완료] {output_file} → {len(chunk)}개 레시피")

[읽기 완료] TB_RECIPE_SEARCH-220701.csv
[읽기 완료] TB_RECIPE_SEARCH-20231130.csv
[읽기 완료] TB_RECIPE_SEARCH_241226.csv


c:\Users\User\miniconda3\envs\boracay\Lib\site-packages\numpy\_core\fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


[저장 완료] recipe_mangae_1.json → 84146개 레시피
[저장 완료] recipe_mangae_2.json → 84146개 레시피
[저장 완료] recipe_mangae_3.json → 84146개 레시피
[저장 완료] recipe_mangae_4.json → 84145개 레시피


### key 값 전처리

- 기존 영어 -> 의미에 맞는 이름으로 변환
- 사이트 제공 참고

| index | pk    | type         | nameEn             | nameKo(제공)        | 최종 이름        |
| ----- | ----- | ------------ | ------------------ | ------------------ | --------------- |
| 0     | Y     | NUMBER(40)   | RCP_SNO            | "레시피일련번호"     |"recipe_id"      |
| 1     | N     | VARCHAR(200) | RCP_TTL            | "레시피제목"         |"recipe_title"   | 
| 2     | N     | VARCHAR(40)  | CKG_NM             | "요리명"            |"dish_name"      |
| 3     | N     | VARCHAR(32)  | RGTR_ID            | "등록자ID"          |"author_id"     | 
| 4     | N     | VARCHAR(64)  | RGTR_NM            | "등록자명"          |"author_name"    |
| 5     | N     | NUMBER(11)   | INQ_CNT            | "조회수"            |"view_count"    |
| 6     | N     | NUMBER(11)   | RCMM_CNT	        | "추천수"            |"like_count"    |
| 7     | N     | NUMBER(11)   | SRAP_CNT           | "스크랩수"          |"bookmark_count" |
| 8     | N     | VARCHAR(200) | CKG_MTH_ACTO_NM    | "요리방법별명"       |"cooking_method"    |
| 9     | N     | VARCHAR(200) | CKG_STA_ACTO_NM    | "요리상황별명"       |"cooking_situation"    |
| 10    | N     | VARCHAR(200) | CKG_MTRL_ACTO_NM   | "요리재료별명"       |"main_ingredient"    |
| 11    | N     | VARCHAR(200) | CKG_KND_ACTO_NM    | "요리종류별명"       |"dish_type"    |
| 12    | N     | VARCHAR(4000)| CKG_IPDC           | "요리소개"          |"description"    |
| 13    | N     | VARCHAR(4000)| CKG_MTRL_CN        | "요리재료내용"       |"ingredients"    |
| 14    | N     | VARCHAR(200) | CKG_INBUN_NM       | "요리인분명"         |"servings"    |
| 15    | N     | VARCHAR(200) | CKG_DODF_NM        | "요리난이도명"       |"difficulty"    |
| 16    | N     | VARCHAR(200) | CKG_TIME_NM        | "요리시간명"         |"cooking_time"    |
| 17    | N     | CHAR(14)	   | FIRST_REG_DT       | "최초등록일시"       |"created_at"    |
| 18    | N     | VARCHAR(200) | RCP_IMG_URL        | *미제공             |"image_url"    |

In [2]:
import logging

# 로거 설정
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# 바꿀 key 매핑
key_map = {
    "RCP_SNO": "recipe_id",
    "RCP_TTL": "recipe_title",
    "CKG_NM": "dish_name",
    "RGTR_ID": "author_id",
    "RGTR_NM": "author_name",
    "INQ_CNT": "view_count",
    "RCMM_CNT": "like_count",
    "SRAP_CNT": "bookmark_count",
    "CKG_MTH_ACTO_NM": "cooking_method",
    "CKG_STA_ACTO_NM": "cooking_situation",
    "CKG_MTRL_ACTO_NM": "main_ingredient",
    "CKG_KND_ACTO_NM": "dish_type",
    "CKG_IPDC": "description",
    "CKG_MTRL_CN": "ingredients",
    "CKG_INBUN_NM": "servings",
    "CKG_DODF_NM": "difficulty",
    "CKG_TIME_NM": "cooking_time",
    "FIRST_REG_DT": "created_at",
    "RCP_IMG_URL": "image_url"
}

for i in range(1,5):
    data=load_json(f"recipe_mangae_{i}.json")
    converted_data=[{key_map.get(k, k): v for k, v in recipe.items()} for recipe in data]
    save_json(converted_data,f"recipe_mangae_{i}.json")
    logger.info(f"recipe_mangae_{i}.json - key 변환 완료")

INFO:__main__:recipe_mangae_1.json - key 변환 완료
INFO:__main__:recipe_mangae_2.json - key 변환 완료
INFO:__main__:recipe_mangae_3.json - key 변환 완료
INFO:__main__:recipe_mangae_4.json - key 변환 완료


- 사용되지 않을 것으로 판단한 key 삭제 
- 등록자 관련 정보 삭제. 레시피를 누가 작성했냐는 사용될 일 없음 -> "author_id", "author_name"
- 주재료 삭제. 재료 안에도 담겨있는 경우(재료 안에서 검색하면 됨) 혹은 '가공식품류', '채소류' 등의 의미 없는 정보. -> "main_ingredient"
- 요리 방법 삭제. '끓이기', '굽기' 등의 간단한 요리 방법은 레시피에 제공할 필요도 없고, ai 검색에도 필요 없을 것으로 판단. -> "cooking_method"

In [ ]:
import os
import json
import logging

# 로깅 설정
logging.basicConfig(
    level=logging.INFO,
    format='[%(asctime)s] %(levelname)s - %(message)s'
)

# 삭제할 키 목록
keys_to_delete = ["author_id", "author_name", "main_ingredient", "cooking_method"]

for i in range(1,5):
    data = load_json(f"recipe_mangae_{i}.json")

    for recipe in data:
        for key in keys_to_delete:
                del recipe[key]

    save_json(data,f"recipe_mangae_{i}.json")
logging.info(f"key 삭제 완료")


[2025-07-06 17:48:51,532] INFO - key 삭제 완료
